In [36]:
%run imports.ipynb

imports loaded.


### Func def

#### pre-processing

In [27]:
print('\n> Pre-Processing func:')


> Pre-Processing func:


In [3]:
def get_height(item):
    
    '''return height in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return round(30.48 * float(item[0:item.find("'")]) + 2.54 * float(item[item.find("'")+1:item.find('"')]),2)

print('    get_height(item)')    

    get_height(item)


In [4]:
def get_weight(item):
    
    '''return weight in kg'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 0.45 * float(item[0:item.find('lbs.')])
    
print('    get_weight(item)')    

    get_weight(item)


In [5]:
def get_reach(item):
    
    '''return reach in cm'''
    
    item = str(item)
    
    if item[0] == '-':
        return None
    else:
        return 2.54 * float(item[0:item.find('"')])
    
print('    get_reach(item)')    

    get_reach(item)


In [6]:
def get_std_reach(item, height, reach_list, height_list):
    '''return "standard reach" ( =mean(reach/height)*height )'''
    return round(height*np.mean(reach_list/height_list),2)

print('    get_std_reach(item, height, reach_list, height_list)')

    get_std_reach(item, height, reach_list, height_list)


In [7]:
def get_age(item):
    
    '''return age (instead of date of birth)'''
    
    from datetime import date,timedelta
    
    item = str(item)
    current_date = date.today().year
    
    if item[0] == '-':
        return None
    else:
        return (current_date - int(item[item.find(',')+1:]))
    
print('    get_age(item)')    

    get_age(item)


In [8]:
def get_std_age(item, age_list):
    
    '''return standrd age per weight class'''  
    return round(np.mean(age_list),2)

print('    get_std_age(item, age_list)')

    get_std_age(item, age_list)


In [9]:
def get_win_prob(name, date, df):
    
    '''return current career winning %'''
    
    wins = len(df[(df['fighter'] == name) & (df.date < date)])
    losses = len(df[(df['opponent'] == name) & (df.date < date)])
    
    if wins == 0:
        return 0
    return wins/(wins+losses)

print('    get_win_prob(name, date, df)')

    get_win_perc(name, date, df)


In [13]:
def get_win_streak(name, date, df):
    
    data = df[(df.fighter==name)|(df.opponent==name)].sort_values(by='date',ascending=False)
    data = data[data.date < date].reset_index(drop=True)
    
    if len(data) < 1:
        return 0
    
    count = 0
    for fighter in data['fighter']:
        if fighter != name:
            return count
        count += 1
    
    return count

print('    get_win_streak(name, date, df) #return count of recent wins')

    get_win_streak(name, date, df)


In [14]:
def get_lose_streak(name, date, df):
    
    data = df[(df.fighter==name)|(df.opponent==name)].sort_values(by='date',ascending=False)
    data = data[data.date < date].reset_index(drop=True)
    
    if len(data) < 1:
        return 0
    
    count = 0
    for opponent in data['opponent']:
        if opponent != name:
            return count
        count += 1
    
    return count

print('    get_lose_streak(name, date, df)')

    get_lose_streak(name, date, df)


In [4]:
def get_fighter_stats(name, data ):
    '''return fighters statistics from stat_df'''
    
    stat_list = data.columns[1:]
    row = (data[stat_list][data.name==name]).reset_index(drop=True)
    
    if len(row) < 1:
        return len(stat_list)*[None]
    else:
        return row.loc[0]
    
print('    get_fighter_stats(name, data )')    

    get_fighter_stats(name, data )


#### feature_engineering

In [28]:
print('\n> Feature Engineering func:')


> Feature Engineering func:


In [18]:
def get_stat(index, data, stat='height', fo=''):
    return data.loc[index, fo + stat]

print("    get_stat(index, data, stat='height', fo='')")

    get_stat(index, data, stat='height', fo='')


In [5]:
def get_swaped_row(index, data):
    
    swapped_row = []
    for column in data.columns:
        if column.find('f_') > -1:
            swapped_row.append(data.loc[index,'o_' + column[2:]])
        elif column.find('o_') > -1:
            swapped_row.append(data.loc[index,'f_' + column[2:]]) 
        elif column == 'result':
            swapped_row.append('lose')
        elif column == 'fighter':
            swapped_row.append(data.loc[index,'opponent'])
        elif column == 'opponent':
            swapped_row.append(data.loc[index,'fighter'])
        else:
            swapped_row.append(data.loc[index,column])    
    
    return swapped_row

print('    get_swaped_row(index, data) # return fighter & opponent statistics swapped. ')

    get_swaped_row(index, data)


In [6]:
def get_wins(combo, data):

    data = data[(data.result=='win') & (data.stance_combo==combo)]
    return len(data)

print('    get_wins(combo, data)')

    get_wins(combo, data)


In [7]:
def get_wins_ratio(combo, data):
    
    combo_df = data[data.stance_combo==combo]
    if len(combo_df) == 0:
        return 0
    else:
        return get_wins(combo, data=data)/len(combo_df)
    
print('    get_wins_ratio(combo, data)')    

    get_wins_ratio(combo, data)


#### outlier_cleaning

In [35]:
print('\n> outlier_cleaning func:')


> outlier_cleaning func:


In [42]:
def set_outlier_detection_model(X_columns, data, n=20, contemination=0.05):
    
    lof = LocalOutlierFactor(n_neighbors=n, algorithm='auto', metric='minkowski',contamination=contemination, novelty=False, n_jobs=-1)
    lof.fit(data[X_columns])
    return lof
print('    set_outlier_detection_model(X_columns, data)')

    set_outlier_detection_model(X_columns, data)


>>>>

#### model_stats_pred_training

In [8]:
print('\n> model_stats_pred_training func:')


> model_stats_pred_training func:


In [9]:
def set_lower_cases(string):
    return ''.join(c.lower() for c in string if c.isalpha())

print('    set_lower_cases(string)')

    set_lower_cases(string)


In [10]:
def set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3):

    if 'set' not in data.columns:
        data.insert(0, 'set', None) # inserd 'set' column to position 0
    
    for i in range(len(data)):
        
        if data.loc[i,'result'] == None:
            data.loc[i,'set'] = 'new'
        
        elif random.random() < test_size:
            data.loc[i,'set'] = 'test'
        else:
            if random.random() < valid_size:
                data.loc[i,'set'] = 'validation'
            else:
                data.loc[i,'set'] = 'train' 

    X_train = data[X_columns][data.set=='train']
    X_test = data[X_columns][data.set=='test']
    y_train = data[y_columns][data.set=='train']
    y_test = data[y_columns][data.set=='test']
    X_val = data[X_columns][data.set=='validation']
    y_val = data[y_columns][data.set=='validation']
    
    print('Data split: Data=%s, train_set=%s, validation_set=%s, test_set=%s'  %(str(data.shape), str(X_train.shape), str(X_val.shape), str(X_test.shape)))
    return data

print('    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)')

    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)


In [11]:
def set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5):
    
    xgboost_reg = XGBRegressor(n_estimators=n, max_depth=max_depth, learning_rate=learning_rate, subsample=subsample)
    X_train = data.loc[data.set=='train',X_columns]
    y_train = np.ravel(np.array(data.loc[data.set=='train',y_columns]).reshape(-1,1))
    xgboost_reg.fit(X_train,y_train)
    return xgboost_reg

print('    set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5)')

    set_xgboost_reg(X_columns, y_columns, data, n=200, learning_rate=0.2, subsample=0.4, max_depth=5)


In [16]:
def set_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column + '_pred'] = None
        data[column + '_pred'] = model.predict(data[X_columns])
    
    return data

print('    set_predictions(model, data, X_columns, y_columns)')

    set_predictions(model, data, X_columns, y_columns)


In [ ]:
def get_mean_percentage_error(act, pred):
    residual = np.abs(act - pred)
    mean = np.mean(act)
    ucl = mean + 3*np.std(act)
    lcl = mean - 3*np.std(act)
    return np.mean(1 - (residual/(ucl-lcl)))

print('    get_mean_percentage_error(act, pred)')

In [ ]:
def save_trained_model(model, filename_path):
    
    with open(filename_path, 'wb') as f:
        pickle.dump(model, f)
    
    print(filename_path + ' saved.')
    
print('    save_trained_model(model, path)')    

#### result_model_training

In [ ]:
print('\n> result_model_training func:')

In [30]:
def set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs'):
    
    X_train = data[data['set'] == 'train'][X_columns]
    y_train = np.ravel(np.array(data[data['set'] == 'train'][y_columns]).reshape(-1,1))
    
    # Create a logistic regression model
    model = LogisticRegression(C=c, solver=solver, penalty=penalty)

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Create a dataframe with the predicted results
    return model

print("    set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs')")

    set_logistic_reg_clf(X_columns, y_columns, data, penalty='elasticnet', c=1, solver='lbfgs')


In [ ]:
def get_accuracy(act, pred):
    
    true = 0
    
    for act_value, pred_value in zip(act, pred):
        if act_value == pred_value:
            true += 1
    
    return true/len(act)    
    #return accuracy_score(act, pred)

print('    get_accuracy(act, pred)')

In [ ]:
def get_mean_accuracy(act, pred):

    return np.mean([get_accuracy(act, pred),get_accuracy(act, pred),get_accuracy(act, pred)])    


print('    get_mean_accuracy(act, pred) #scoring performance x3 times and return its mean.')

In [ ]:
def get_model_evaluation(data,y_column):

    y_test_act = data[y_column][data.set=='test']
    y_test_pred = data[y_column + '_pred'][data.set=='test']

    print('Accuracy = %.6f' % accuracy_score(y_test_act,y_test_pred))
    ConfusionMatrixDisplay.from_predictions(y_test_act,y_test_pred) 
    #return accuracy_score(y_test_act,y_test_pred)
    
print('    get_model_evaluation(data)')    

In [ ]:
def load_trained_model(file_path):
    with open(file_path, 'rb') as file:
        model = pickle.load(file)
        
    return model

print('    load_trained_model(file_path) #.PKL model from ../global_variables/')

#### method_model_training

In [37]:
print('\n> method_model_training func:')


> method_model_training func:


In [36]:
def set_knn_clf(X_columns, y_columns, data, neighbors=5):
    
    X_train = data[data['set'] == 'train'][X_columns]
    y_train = np.ravel(np.array(data[data['set'] == 'train'][y_columns]).reshape(-1,1))
    
    # Create a KNN classifier model
    model = KNeighborsClassifier(n_neighbors=neighbors)

    # Fit the model to the training data
    model.fit(X_train, y_train)

    # Create a dataframe with the predicted results
    return model

print("    set_knn_clf(X_columns, y_columns, data, neigbors=5)")

    set_knn_clf(X_columns, y_columns, data, neigbors=5)


#### predictor

In [31]:
print('\n> Predictor func:')


> Predictor func:


In [39]:
def get_current_date():
    return date.today().strftime('%Y-%m-%d')

print('    get_current_date()')

    get_current_date()


In [40]:
def get_win_perc(fighter, data):
    '''return current career winning %'''
    date = get_current_date()
    wins = len(data[(data['fighter'] == fighter)&(data.date < date)&(data['result']=='win')]) + len(data[(data['opponent'] == fighter)&(data.date < date)&(data['result']=='lose')])
    losses = len(data[(data['fighter'] == fighter)&(data.date < date) & (data['result']=='lose')]) + len(data[(data['opponent'] == fighter)&(data.date < date) & (data['result']=='win')])
    
    if wins == 0:
        return 0
    return round(wins/(wins+losses),3)

print('    get_win_perc(fighter, data)')

    get_win_perc(fighter, data)


In [ ]:
def get_win_streak(fighter, data):
    '''return current win streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'win')&(f == fighter))|((result == 'lose')&(o == fighter)):
            count += 1
        else:
            return count
    return count

print('    get_win_streak(fighter, data)')

In [41]:
def get_lose_streak(fighter, data):
    '''return current lose streak'''
    date = get_current_date()
    data = ((data[(data.date < date)&((data.fighter == fighter)|(data.opponent == fighter))]).sort_values(by=['date'], ascending=False)).reset_index(drop=True)
    
    count = 0
    for i in range(0,len(data)):
        
        result = data.loc[i,'result']
        f = data.loc[i,'fighter']
        o = data.loc[i,'opponent']
        
        if ((result == 'lose')&(f == fighter))|((result == 'win')&(o == fighter)):
            count += 1
        else:
            return count
    return count

print('    get_lose_streak(fighter, data)')

    get_lose_streak(fighter, data)


In [46]:
def get_stance_wins_ratio(fighter, data):
    try:
        return np.mean(data.stance_win_ratio[data.fighter == fighter].reset_index(drop=True).iloc[0])
    except:
        return 0

print('    get_stance_wins_ratio(fighter, data)')

    get_stance_wins_ratio(fighter, data)


In [ ]:
>>>>

def get_td_avg(fighter, data):
    '''return current takedown average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td avg.']

print('    get_td_avg(fighter, data)')

def get_td_acc(fighter, data):
    '''return current takedown accuracy'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td acc.']

print('    get_td_acc(fighter, data)')

def get_td_def(fighter, data):
    '''return current takedown defense'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'td def.']

print('    get_td_def(fighter, data)')

def get_str_acc(fighter, data):
    '''return current striking accuracy'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'str. acc.']

print('    get_str_acc(fighter, data)')

def get_str_def(fighter, data):
    '''return current striking defense'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'str. def']

print('    get_str_def(fighter, data)')

def get_sub_avg(fighter, data):
    '''return current submission average'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sub. avg.']

print('    get_sub_avg(fighter, data)')

def get_sapm(fighter, data):
    '''return current strikes absorbed per minute'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'sapm']

print('    get_sapm(fighter, data)')

def get_slpm(fighter, data):
    '''return current strikes landed per minutes'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'slpm']

print('    get_slpm(fighter, data)')

def get_stance(fighter, data):
    '''return fighter stance'''
    data = (data[data.name == fighter]).reset_index(drop=True)
    return data.loc[0,'stance']

print('    get_stance(fighter, data)')

In [ ]:
def set_new_fight(fighter, opponent, data):
    
    row = {'set': None,
           'method': None,
           'date': get_current_date(),
           'result': None,
           'fighter': fighter,
           'opponent': opponent,
           'kd_dif':None,
           'str_dif':None,
           'td_dif':None,
           'sub_dif':None,
           
           'td avg._dif': get_td_avg(fighter=fighter, data=stats) - get_td_avg(fighter=opponent, data=stats),
           'td acc._dif': get_td_acc(fighter=fighter, data=stats) - get_td_acc(fighter=opponent, data=stats),
           'td def._dif': get_td_def(fighter=fighter, data=stats) - get_td_def(fighter=opponent, data=stats),
           'sub. avg._dif': get_sub_avg(fighter=fighter, data=stats) - get_sub_avg(fighter=opponent, data=stats),
           'slpm_dif': get_slpm(fighter=fighter, data=stats) - get_slpm(fighter=opponent, data=stats),
           'str. acc._dif': get_str_acc(fighter=fighter, data=stats) - get_str_acc(fighter=opponent, data=stats),
           'sapm_dif': get_sapm(fighter=fighter, data=stats) - get_sapm(fighter=opponent, data=stats),
           'str. def_dif': get_str_def(fighter=fighter, data=stats) - get_str_def(fighter=opponent, data=stats),
           
           'stance_win_ratio':get_stance_wins_ratio(fighter=fighter, data=stats) - get_stance_wins_ratio(fighter=opponent, data=stats),
           'win_perc_dif': get_win_perc(fighter=fighter, data=data) - get_win_perc(fighter=opponent, data=data),
           'win_strk_dif': get_win_streak(fighter=fighter, data=data) - get_win_streak(fighter=opponent, data=data),
           'lose_strk_dif': get_lose_streak(fighter=fighter, data=data) - get_lose_streak(fighter=opponent, data=data),       
          }
    return pd.DataFrame(row, index=[0])

print('    set_new_fight(fighter, opponent, data)')

def set_knn_predictor(X_columns, y_columns, data, n=5):
    
    knn_reg = KNeighborsRegressor(n_neighbors=n)
    X_train = data[X_columns][data.set!='new']
    y_train = data[y_columns][data.set!='new']
    knn_reg.fit(X_train,y_train)
    return knn_reg
    
print('    set_knn_predictor(X_columns, y_columns, data, n=5)')  

def set_knn_predictions(model, data, X_columns, y_columns):
    
    for column in y_columns:
        data[column] = None
        data[column] = model.predict(data[X_columns])
    
    return data

print('    set_knn_predictions(model, data, X_columns, y_columns)')

def set_knn_clf_predictor(X_columns, y_columns, data, neighbors):
    
    knn_clf = KNeighborsClassifier(n_neighbors=neighbors, weights='distance')
    X_train = data[X_columns][data.set!='new']
    y_train = data[y_columns][data.set!='new']
    
    if len(y_columns) == 1:
        y_train = np.ravel(np.array(data[y_columns[0]][data.set!='new']).reshape(-1,1))
    else:    
        y_train = data[y_columns][data.set!='new']
    
    knn_clf.fit(X_train,y_train)
    return knn_clf

print("    set_knn_clf_predictor(X_columns, y_columns, data, neighbors)")        

def set_pred_results(model, data, X_columns, y_columns):
    
    data['result_pred'] = None
    data['result_pred'] = model.predict(data[X_columns])    
    return data

print('    set_pred_results(model, data, X_columns, y_columns)')

def get_fighters_stats(fighter, opponent, data):
    
    data1 = pd.DataFrame((data[data.name == fighter]).reset_index(drop=True).loc[0])[1:].rename(columns={0:fighter})
    data2 = pd.DataFrame((data[data.name == opponent]).reset_index(drop=True).loc[0])[1:].rename(columns={0:opponent})
    return pd.concat([data1,data2], axis=1)

print('    get_fighters_stats(fighter, opponent, data)')

def get_winner(match):
    '''return winner of the fight'''
    
    result = match.loc[0,'result']
   
    if result == 'win':
        winner = fighter
    else: 
        winner = opponent    
    
    return winner

print('    get_winner(match)')

#### micro_model_training

print('\n> micro_model_training func:')

def set_result_column(fighter,data):
    
    for i in range(len(data)):
        if (data.loc[i,'result'] == 'win')&(data.loc[i,'fighter'] == fighter):
            data.loc[i,'result'] = fighter + ' win'
        elif (data.loc[i,'result'] == 'lose')&(data.loc[i,'opponent'] == fighter):
            data.loc[i,'result'] = fighter + ' win'
        else:
            data.loc[i,'result'] = fighter + ' lose'
    return data

print('    set_result_column(fighter,data)')

def get_my_losses(fighter, data):
    data = data[data.result == fighter + ' lose']
    return data

print('    get_my_losses(fighter, data)')

def get_my_wins(fighter, data):
    data = data[data.result == fighter + ' win']
    return data

print('    get_my_wins(fighter, data)')

def get_synt_loss(fighter):
    
    '''creating artificial random lossess for fighter who is undefeated'''
    
    data = pd.read_csv('../data/fight_stat_data_trained.csv').drop(columns=['Unnamed: 0'])
    data = data[data.result == 'lose'].reset_index(drop=True)

    row = {'result':fighter + ' lose',
           'fighter':fighter,
           'opponent':'Anonymous',
           'date':get_current_date(),
           'weight class':None,
           'method':None,
           'kd_dif':get_loss_value(values=data['kd_dif']),
           'str_dif':get_loss_value(values=data['str_dif']),
           'td_dif':get_loss_value(values=data['td_dif']),
           'sub_dif':get_loss_value(values=data['sub_dif']),
           'td avg._dif':get_loss_value(values=data[ 'td avg._dif']),
           'td acc._dif':get_loss_value(values=data['td acc._dif']),
           'td def._dif':get_loss_value(values=data['td def._dif']),
           'sub. avg._dif':get_loss_value(values=data[ 'sub. avg._dif']),
           'slpm_dif':get_loss_value(values=data['slpm_dif']),
           'str. acc._dif':get_loss_value(values=data[ 'str. acc._dif']),
           'sapm_dif':get_loss_value(values=data['sapm_dif']),
           'str. def_dif':get_loss_value(values=data['str. def_dif']),
           'stance_win_ratio':get_loss_value(values=data[ 'stance_win_ratio']),
           'win_perc_dif':get_loss_value(values=data['win_perc_dif']),
           'win_strk_dif':get_loss_value(values=data[ 'win_strk_dif']),
           'lose_strk_dif':get_loss_value(values=data[ 'lose_strk_dif']),
           'set':'train'
          }
    return pd.DataFrame(data=row, index=[0])

print("    get_synt_loss(fighter)")

 def get_loss_value(values):
    '''return value in normal range'''
    mean = np.mean(values)
    std = np.std(values) 
    lower_bound = mean - std
    upper_bound = mean + std
    return random.uniform(lower_bound, upper_bound)

print('    get_loss_value(values)')